**LLM Workshop 2024 by Sebastian Raschka**

This code is based on *Build a Large Language Model (From Scratch)*, [https://github.com/rasbt/LLMs-from-scratch](https://github.com/rasbt/LLMs-from-scratch)

<br>
<br>
<br>
<br>

# 5) Loading pretrained weights (part 1)

In [1]:
from importlib.metadata import version

pkgs = ["matplotlib", 
        "numpy", 
        "tiktoken", 
        "torch",
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")

matplotlib version: 3.9.4
numpy version: 1.26.4
tiktoken version: 0.9.0
torch version: 2.5.1


- Previously, we only trained a small GPT-2 model using a very small short-story book for educational purposes
- Fortunately, we don't have to spend tens to hundreds of thousands of dollars to pretrain the model on a large pretraining corpus but can load pretrained weights (we start with the GPT-2 weights provided by OpenAI)

<img src="figures/01.png" width=1000px>

- First, some boilerplate code to download the files from OpenAI and load the weights into Python
- Since OpenAI used [TensorFlow](https://www.tensorflow.org/), we will have to install and use TensorFlow for loading the weights; [tqdm](https://github.com/tqdm/tqdm) is a progress bar library
- Uncomment and run the next cell to install the required libraries

In [ ]:
# pip install tensorflow tqdm

In [2]:
print("TensorFlow version:", version("tensorflow"))
print("tqdm version:", version("tqdm"))

TensorFlow version: 2.19.0
tqdm version: 4.67.1


In [ ]:
from gpt_download import download_and_load_gpt2
# The download_and_load_gpt2 function will download the relevent files from OpenAI website that we will load into the model.

- We can then download the model weights for the 124 million parameter model as follows:

In [4]:
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 10.1kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:05<00:00, 188kiB/s] 
hparams.json: 100%|██████████| 90.0/90.0 [00:00<00:00, 16.5kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [02:57<00:00, 2.80MiB/s]   
model.ckpt.index: 100%|██████████| 5.21k/5.21k [00:00<00:00, 1.18MiB/s]
model.ckpt.meta: 100%|██████████| 471k/471k [00:03<00:00, 146kiB/s]  
vocab.bpe: 100%|██████████| 456k/456k [00:01<00:00, 231kiB/s]  


In [ ]:
print("Settings:", settings)
# The settings dictionary contains the configuration of the model. We will load these settings into the model.

Settings: {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}


In [ ]:
print("Parameter dictionary keys:", params.keys())
# The parameter dictionary contains the weights of the model. We will use these weights to load it into the model.
# 'blocks' contains the weights of the transformer blocks
# 'b' & 'g' are the variables for the biases and the weights of the Final LayerNorm
# 'wpe' is the position embedding layer
# 'wte' is the token embedding layer

Parameter dictionary keys: dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])


In [14]:
print(len(params['blocks'])) # = 12 i.e the number of transformer blocks in the model
print(params['blocks'][0].keys()) # The keys of the first transformer block where ln_1 is the first layer norm, ln_2 is the second layer norm, attn is the Masked multi-head attention layer, mlp is the Feed Forward layer
print(params['blocks'][0]['mlp'].keys()) # The keys of the Feed Forward layer are:  c_fc is the first linear layer, c_proj is the second linear layer and act is the activation function which doesn't accept any parametes so, it is not included in the params dictionary.
print(params['blocks'][0]['mlp']['c_fc'].keys()) # The keys of the 'c_fc' (i.e 1st linear layer) are: 'weight' (i.e w) and 'bias' (i.e b) which are the weights and biases of the layer.
print("Weights: \n", params['blocks'][0]['mlp']['c_fc']['w'])
print("Shape of Weights: \n", params['blocks'][0]['mlp']['c_fc']['w'].shape)
# Now, we have to load these weights into our model.


12
dict_keys(['attn', 'ln_1', 'ln_2', 'mlp'])
dict_keys(['c_fc', 'c_proj'])
dict_keys(['b', 'w'])
Weights: 
 [[ 0.09420195  0.09824043 -0.03210221 ... -0.17830218  0.14739013
   0.07064556]
 [-0.12645245 -0.06710183  0.03051439 ...  0.19662377 -0.1203471
  -0.06284904]
 [ 0.04961119 -0.03729444 -0.04833835 ...  0.06554844 -0.07141103
   0.08258601]
 ...
 [ 0.0480442   0.1574535   0.00142291 ... -0.3986896   0.08889695
   0.02402452]
 [ 0.03240572  0.12492938 -0.04256118 ... -0.19337358  0.12716232
  -0.04046172]
 [-0.0316235   0.00099418 -0.04906889 ... -0.0406176   0.05363071
   0.18956499]]
Shape of Weights: 
 (768, 3072)


<img src="../03_architecture/figures/03.png" width="1200px">

In [15]:
print(params["wte"])
print("Token embedding weight tensor dimensions:", params["wte"].shape)

[[-0.11010301 -0.03926672  0.03310751 ... -0.1363697   0.01506208
   0.04531523]
 [ 0.04034033 -0.04861503  0.04624869 ...  0.08605453  0.00253983
   0.04318958]
 [-0.12746179  0.04793796  0.18410145 ...  0.08991534 -0.12972379
  -0.08785918]
 ...
 [-0.04453601 -0.05483596  0.01225674 ...  0.10435229  0.09783269
  -0.06952604]
 [ 0.1860082   0.01665728  0.04611587 ... -0.09625227  0.07847701
  -0.02245961]
 [ 0.05135201 -0.02768905  0.0499369  ...  0.00704835  0.15519823
   0.12067825]]
Token embedding weight tensor dimensions: (50257, 768)


- Alternatively, "355M", "774M", and "1558M" are also supported `model_size` arguments
- The difference between these differently sized models is summarized in the figure below:

<img src="figures/02.png" width=800px>

- Above, we loaded the 124M GPT-2 model weights into Python, however we still need to transfer them into our `GPTModel` instance
- First, we initialize a new GPTModel instance
- Note that the original GPT model initialized the linear layers for the query, key, and value matrices in the multi-head attention module with bias vectors, which is not required or recommended; however, to be able to load the weights correctly, we have to enable these too by setting `qkv_bias` to `True` in our implementation, too
- We are also using the `1024` token context length that was used by the original GPT-2 model(s)

In [16]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}


# Define model configurations in a dictionary for compactness
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# Copy the base configuration and update with specific model settings
model_name = "gpt2-small (124M)"  # We choose the 124Million GPT2 model
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})
print(NEW_CONFIG)

{'vocab_size': 50257, 'context_length': 1024, 'emb_dim': 768, 'n_heads': 12, 'n_layers': 12, 'drop_rate': 0.1, 'qkv_bias': True}


In [19]:
from supplementary import GPTModel

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

- The next task is to assign the OpenAI weights to the corresponding weight tensors in our `GPTModel` instance

In [ ]:
import torch

# This is a utility function which converts the torch tensor to a torch parameter if left tensor = right tensor
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))  # converting the torch tensor to a torch parameter

In [21]:
import numpy as np

# The 'load_weights_into_gpt' function is responsible for loading pre-trained weights into our 'gpt' model

def load_weights_into_gpt(gpt, params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])
    
    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.weight = assign(
            gpt.trf_blocks[b].att.W_query.weight, q_w.T)
        gpt.trf_blocks[b].att.W_key.weight = assign(
            gpt.trf_blocks[b].att.W_key.weight, k_w.T)
        gpt.trf_blocks[b].att.W_value.weight = assign(
            gpt.trf_blocks[b].att.W_value.weight, v_w.T)

        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].att.W_query.bias = assign(
            gpt.trf_blocks[b].att.W_query.bias, q_b)
        gpt.trf_blocks[b].att.W_key.bias = assign(
            gpt.trf_blocks[b].att.W_key.bias, k_b)
        gpt.trf_blocks[b].att.W_value.bias = assign(
            gpt.trf_blocks[b].att.W_value.bias, v_b)

        gpt.trf_blocks[b].att.out_proj.weight = assign(
            gpt.trf_blocks[b].att.out_proj.weight, 
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].att.out_proj.bias = assign(
            gpt.trf_blocks[b].att.out_proj.bias, 
            params["blocks"][b]["attn"]["c_proj"]["b"])

        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight, 
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias, 
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight, 
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias, 
            params["blocks"][b]["mlp"]["c_proj"]["b"])

        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale, 
            params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift, 
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale, 
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift, 
            params["blocks"][b]["ln_2"]["b"])

    gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
    gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])
    
    
load_weights_into_gpt(gpt, params)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt.to(device);

- If the model is loaded correctly, we can use it to generate new text using our previous `generate` function:

In [24]:
import tiktoken
from supplementary import (
    generate_text_simple,
    text_to_token_ids,
    token_ids_to_text
)


tokenizer = tiktoken.get_encoding("gpt2")

torch.manual_seed(123)

input_text = "Every effort moves you"
input_token_ids = text_to_token_ids(input_text, tokenizer).to(device)
output_token_ids = generate_text_simple(
    model=gpt,
    idx=input_token_ids,
    max_new_tokens=50,
    context_size=NEW_CONFIG["context_length"]
)

print("Output text:\n", token_ids_to_text(output_token_ids, tokenizer))

Output text:
 Every effort moves you forward.

The first step is to understand the importance of your work.

The second step is to understand the importance of your work.

The third step is to understand the importance of your work.

The fourth step is


- We know that we loaded the model weights correctly because the model can generate coherent text; if we made even a small mistake, the mode would not be able to do that

<br>
<br>
<br>
<br>

# Exercise 1: Trying larger LLMs

- Load one of the larger LLMs and see how the output quality compares
- Ask it to answer specific instructions, for example to summarize text or correct the spelling of a sentence